### 14 May 2021
# Jordan C. - COMP3700 - Final Project


In [6]:
# initialize Spark session
import os, sys, pyspark.sql.functions as f
top_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
if top_dir not in sys.path:
    sys.path.append(top_dir)
from init_spark import init_spark
spark = init_spark()

Initializing Spark...
Spark found in :  /home/ubuntu/apps/spark
Spark config:
	 executor.memory=2g
	some_property=some_value
	spark.app.name=TestApp
	spark.master=local[*]
	spark.sql.warehouse.dir=/tmp/tmpodw4ztbv
	spark.submit.deployMode=client
	spark.submit.pyFiles=
	spark.ui.showConsoleProgress=true
Spark UI running on port 4041


In [7]:
# load Chipotle orders data
chipotleOrdersDF0 = spark.read.csv("./data/chipotle-orders.tsv", sep=r'\t', header=True).selectExpr('order_id','quantity','item_name','choice_description','item_price as item_price0')
chipotleOrdersDF0.printSchema()

# convert item_price from string to double and create a table
chipotleOrdersDF1 = chipotleOrdersDF0.withColumn('item_price', f.regexp_replace('item_price0', '[$,]', '').cast('double')).drop('item_price0')
chipotleOrdersDF1.createOrReplaceTempView("chipotle_orders")
chipotleOrdersDF1.printSchema()
chipotleOrdersDF1.show(11)

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price0: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: double (nullable = true)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Mild Fr...|                NULL|      2.39|
|       1|       1|                Izze|        [Clementine]|      3.39|
|       1|       1|    Nantucket Nectar|             [Apple]|      3.39|
|       1|       1|Chips and Tomatil...|                NULL|      2.39|
|       2|       2|        Chicken Bowl|[Tomatillo

In [8]:
# group by order ID and sort by order total
orderTotals = spark.sql("select order_id, cast(round(sum(item_price),2) as decimal(10,2)) as order_total from chipotle_orders group by order_id order by order_total desc")
print("TOP 10 orders:")
orderTotals.show(10)

orderTotals.createOrReplaceTempView("order_totals")
chipotleOrdersDF2 = spark.sql("select chipotle_orders.*, order_totals.order_total from chipotle_orders join order_totals on (chipotle_orders.order_id=order_totals.order_id)")
chipotleOrdersDF2.printSchema()

TOP 10 orders:
+--------+-----------+
|order_id|order_total|
+--------+-----------+
|     926|     205.25|
|    1443|     160.74|
|    1483|     139.00|
|     691|     118.25|
|    1786|     114.30|
|     205|     109.90|
|     511|     104.59|
|     491|     102.00|
|    1449|      95.39|
|     759|      86.30|
+--------+-----------+
only showing top 10 rows

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: double (nullable = true)
 |-- order_total: decimal(10,2) (nullable = true)



In [9]:
# create item categories table and join
items = spark.sql("select distinct item_name from chipotle_orders order by item_name")
menuCategories = spark.read.csv("./data/chipotle-menu.tsv", sep=r'\t', header=True).selectExpr('menu_category','item_name','protein','entree_type')
menuCategories.createOrReplaceTempView("menu_categories")
menuCategories = spark.sql("select * from menu_categories order by menu_category, protein, entree_type, item_name")
print("Menu:")
menuCategories.show(50, False)
chipotleOrdersDF = spark.sql("select chipotle_orders.*, menu_categories.menu_category, menu_categories.protein, menu_categories.entree_type from chipotle_orders join menu_categories on (chipotle_orders.item_name=menu_categories.item_name)")
chipotleOrdersDF.createOrReplaceTempView("chipotle_orders")
chipotleOrdersDF.printSchema()
chipotleOrdersDF.show(20, False)

Menu:
+-------------+-------------------------------------+--------+-----------+
|menu_category|item_name                            |protein |entree_type|
+-------------+-------------------------------------+--------+-----------+
|drink        |6 Pack Soft Drink                    |NULL    |NULL       |
|drink        |Bottled Water                        |NULL    |NULL       |
|drink        |Canned Soft Drink                    |NULL    |NULL       |
|drink        |Izze                                 |NULL    |NULL       |
|drink        |Nantucket Nectar                     |NULL    |NULL       |
|entree       |Barbacoa Bowl                        |barbacoa|bowl       |
|entree       |Barbacoa Burrito                     |barbacoa|burrito    |
|entree       |Barbacoa Crispy Tacos                |barbacoa|crispy     |
|entree       |Barbacoa Salad Bowl                  |barbacoa|salad      |
|entree       |Barbacoa Soft Tacos                  |barbacoa|soft       |
|entree       |Carn

In [10]:
#What is the average order total?
spark.sql("select cast(round(avg(order_total), 2) as decimal(10,2)) as AverageOrderTotal from order_totals").show()

+-----------------+
|AverageOrderTotal|
+-----------------+
|            18.81|
+-----------------+



In [11]:
#What is the total revenue generated?
spark.sql("select cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders").show(50, False)

+------------+
|TotalRevenue|
+------------+
|34500.16    |
+------------+



In [12]:
#What is the total revenue by menu item?
spark.sql("select item_name as MenuItem, cast(sum(quantity) as decimal(10,0)) as Qty, cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders group by item_name order by TotalRevenue desc").show(50, False)

+-------------------------------------+---+------------+
|MenuItem                             |Qty|TotalRevenue|
+-------------------------------------+---+------------+
|Chicken Bowl                         |761|7342.73     |
|Chicken Burrito                      |593|5590.62     |
|Steak Burrito                        |388|3866.23     |
|Steak Bowl                           |222|2267.59     |
|Chips and Guacamole                  |506|2201.04     |
|Chicken Salad Bowl                   |123|1228.75     |
|Chicken Soft Tacos                   |120|1108.09     |
|Veggie Burrito                       |97 |934.77      |
|Barbacoa Burrito                     |93 |909.55      |
|Veggie Bowl                          |87 |867.99      |
|Carnitas Bowl                        |74 |758.91      |
|Chips                                |340|680.24      |
|Barbacoa Bowl                        |66 |672.36      |
|Carnitas Burrito                     |60 |597.83      |
|Canned Soft Drink             

In [13]:
#Which protein option is most popular?
spark.sql("select protein as ProteinOption, cast(sum(quantity) as decimal(10,0)) as Qty, cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders where menu_category='entree' group by protein order by Qty desc").show(10, False)

+-------------+----+------------+
|ProteinOption|Qty |TotalRevenue|
+-------------+----+------------+
|chicken      |1659|15845.61    |
|steak        |738 |7413.86     |
|veggie       |217 |2118.65     |
|barbacoa     |206 |2058.98     |
|carnitas     |189 |1885.97     |
+-------------+----+------------+



In [14]:
#Which entree type is most popular?
spark.sql("select entree_type as EntreeType, cast(sum(quantity) as decimal(10,0)) as Qty, cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders where menu_category='entree' group by entree_type order by Qty desc").show(10, False)

+----------+----+------------+
|EntreeType|Qty |TotalRevenue|
+----------+----+------------+
|bowl      |1241|12253.17    |
|burrito   |1231|11899.00    |
|soft      |249 |2344.50     |
|salad     |179 |1775.47     |
|crispy    |109 |1050.93     |
+----------+----+------------+



In [15]:
#Which drink option is most popular?
spark.sql("select first(item_name) as Drink, choice_description as Flavor, cast(sum(quantity) as decimal(10,0)) as Qty, cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders where menu_category='drink' group by choice_description order by Qty desc").show(50, False)

+-----------------+-------------------------+---+------------+
|Drink            |Flavor                   |Qty|TotalRevenue|
+-----------------+-------------------------+---+------------+
|Bottled Water    |NULL                     |211|302.56      |
|Canned Soft Drink|[Diet Coke]              |159|326.71      |
|Canned Soft Drink|[Coke]                   |143|288.79      |
|Canned Soft Drink|[Sprite]                 |89 |133.93      |
|Canned Soft Drink|[Lemonade]               |36 |55.48       |
|Canned Soft Drink|[Coca Cola]              |32 |34.88       |
|Canned Soft Drink|[Dr. Pepper]             |21 |22.89       |
|Canned Soft Drink|[Nestea]                 |20 |35.48       |
|Canned Soft Drink|[Mountain Dew]           |18 |19.62       |
|Canned Soft Drink|[Diet Dr. Pepper]        |14 |15.26       |
|Nantucket Nectar |[Peach Orange]           |9  |30.51       |
|Nantucket Nectar |[Pineapple Orange Banana]|9  |30.51       |
|Izze             |[Clementine]             |8  |27.12 

In [16]:
#Which side option is most popular?
spark.sql("select item_name as Side, cast(sum(quantity) as decimal(10,0)) as Qty, cast(round(sum(item_price), 2) as decimal(10,2)) as TotalRevenue from chipotle_orders where menu_category='side' group by item_name order by Qty desc").show(50, False)

+-------------------------------------+---+------------+
|Side                                 |Qty|TotalRevenue|
+-------------------------------------+---+------------+
|Chips and Guacamole                  |506|2201.04     |
|Chips                                |340|680.24      |
|Chips and Mild Fresh Tomato Salsa    |131|364.36      |
|Chips and Tomatillo-Green Chili Salsa|78 |211.62      |
|Chips and Tomatillo-Red Chili Salsa  |75 |207.25      |
|Chips and Roasted Chili-Corn Salsa   |41 |110.87      |
+-------------------------------------+---+------------+

